# 1. Package installation
These commands should be executed once at the beginning in order to install the packages required to run the model.

In [ ]:
#package installation
!pip install transformers
!pip install pandas

# 2. Import Packages
Run these comands in the beginning in order to import the different packages in the code.

In [73]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2TokenizerFast, GPT2LMHeadModel
import random
from transformers import set_seed
import pandas as pd

# 3. Model parameters
You can set different parameters for the GPT-2 model as follows:
1. 'model_name': GPT-2 comes in different sizes based on the number of parameters it was trained on:
    - 'gpt2': 117M parameters
    - 'gpt2-medium': 345M parameters
    - 'gpt2-large': 774M parameters
    - 'gpt2-xl': 1558M parameters

    Note that the larger the model, the longer it takes to load and use for generation.

2. 'num_samples': the number of sample responses to generate

3. 'max_length': the maximum number of words to be generated in a response.

4. 'sampling': The model generates the response word by word, conditioned on the input text and the sequence of words generated so far in the respose. There are different sampling strategies such as:
    - 'top-p' (nucleas sampling)
    - 'top-k' 
    - 'temperature'
    
    You can also combine different sampling approaches, but for simplicity, we will skip that. This is a nice blog post on sampling: https://huggingface.co/blog/how-to-generate

5. After choosing the 'sampling' strategy, you can set the corresponding value: 'top-p', 'top-k' or 'temperature', where 0 $\leqslant$ top-p $\leqslant$ 1, top-k $\geqslant$ 0, and 0 $\leqslant$ temperature $\leqslant$ 1.

In [96]:
params = {
    'model_name': 'gpt2-medium',
    'num_samples': 1,
    'max_length': 100,
    'sampling': 'top-p',
    'top-p': 0.92,
    'top-k': 40,
    'temperature': 0.7,
    
}

# 4. Loading the model
The following command loads the model into the memory

In [75]:
#load the model
set_seed(42)
tokenizer = GPT2TokenizerFast.from_pretrained(params['model_name'])
model = GPT2LMHeadModel.from_pretrained(params['model_name'], pad_token_id=tokenizer.eos_token_id)

# 5. Reading the input
we can input the text that GPT-2 will response to either by command line:

In [93]:
#generation with interactive mode
prompts = []
prompt = input()
prompts.append(prompt)
prompts

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


['In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.']

or from a text file (input.txt) where each line contains a different prompt:

In [82]:
#generating from a text file
prompts = []
with open('input.txt') as f:
    for line in f:
        prompt = line.strip()
        if prompt == '':
            continue
        prompts.append(prompt)

# 6. Generating the output
By running this code you can generate the GPT-2 responses corresponding to the provided prompts:

In [102]:
#generation itself
output = {}
for text in prompts:
	input_ids = tokenizer.encode(text, return_tensors='pt')
	max_length = len(text.split()) + params['max_length']
	responses = []
	if params['sampling'] == 'top-k':
		responses = model.generate(input_ids, max_length = max_length, do_sample=True, 
            top_k=params['top-k'], num_return_sequences=params['num_samples'])
	elif params['sampling'] == 'temperature':
		responses = model.generate(input_ids, max_length = max_length, do_sample=True, 
            temperature=params['temperature'], num_return_sequences=params['num_samples'])
    #default is nuclear (top-p)
	else:
		responses = model.generate(input_ids, max_length = max_length, do_sample=True, 
            top_p=params['top-p'], num_return_sequences=params['num_samples'])
# 	responses = model.generate(input_ids, max_length=max_length)
	responses = responses[:, input_ids.shape[-1]:]
	final_responses = []
	output[text] = []
	for i, r in enumerate(responses):
		response = tokenizer.decode(r, skip_special_tokens=True)#.strip().split('\n')[0]
		output[text].append(response)
#df = pd.DataFrame(output)
#df
for k in output:
    print('prompt: ' + k)
    for i, r in enumerate(output[k]):
        print('==================================Sample ' + str(i+1), '==================================')
        print(r.strip())
    print('********************************************************************************************************************')
    print('')

prompt: In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
==================================Sample 1 ==================================
The team found that this strange human's words and language work by giving them the information that makes their voices sound like sounds that humans would make if spoken directly. Their findings have implications for future research on language, in which all spoken words must start with a syllable in order to sound like sounds.

According to the research, the humans can learn languages faster than non-human primates, and that's a big deal for humans. "
********************************************************************************************************************

